In [8]:
import cv2
import pickle
import numpy as np
import tensorflow as tf

In [9]:
modelo = tf.keras.models.load_model('modelo_cnn')
modelo.load_weights('pesos_cnn.h5')

In [2]:
with open('modelo_cnn_5clases.pkl', 'rb') as archivo:
    modelo = pickle.load(archivo)

In [13]:
# clases = ['Abraham_Danilo_Miranda_Lopez', 'Alejandro_Benjamin_Rocano_Lopez', 'Alex_Ignacio_Tigselema_Pacheco', 'andres', 'Andres_Patricio_Tapia_Gonzalez', 'Bryan_Eduardo_Martinez_Nunez', 'Christian_Marcelo_Jaramillo_Collazo', 'Diana_Jazmin_Pinchao_Fante', 'Diego_Fernando_Cata_Saltos', 'Edison_Orlando_Lopez_Galarza', 'Jeniffer_Paulina_Yaguana_Caraguay', 'Jonathan_Jose_Pazmino_Fiallos', 'Luis_Fernando_Zerna_Ramos', 'Marco_Farid_Ruano_Caicedo', 'mateo', 'Pablo_Jhoel_Puetate_Mier', 'Silvia_Esthefania_Villacres_Chico', 'Thais_Eliana_Armijos_Troya']
# clases = ['Abraham_Danilo_Miranda_Lopez', 'andres', 'Bryan_Eduardo_Martinez_Nunez', 'Luis_Fernando_Zerna_Ramos', 'mateo']
# clases = ['Abraham_Danilo_Miranda_Lopez', 'andres', 'mateo']
# clases = ['Abraham_Danilo_Miranda_Lopez', 'Alejandro_Benjamin_Rocano_Lopez', 'Alex_Ignacio_Tigselema_Pacheco', 'andres', 'mateo']
clases = ['Abraham_Danilo_Miranda_Lopez', 'Alejandro_Benjamin_Rocano_Lopez', 'Alex_Ignacio_Tigselema_Pacheco', 'andres', 'Andres_Patricio_Tapia_Gonzalez', 'Bryan_Eduardo_Martinez_Nunez', 'Christian_Marcelo_Jaramillo_Collazo', 'Diana_Jazmin_Pinchao_Fante', 'Diego_Fernando_Cata_Saltos', 'Edison_Orlando_Lopez_Galarza', 'Jeniffer_Paulina_Yaguana_Caraguay', 'Jonathan_Jose_Pazmino_Fiallos', 'Luis_Fernando_Zerna_Ramos', 'Marco_Farid_Ruano_Caicedo', 'mateo', 'Pablo_Jhoel_Puetate_Mier', 'Silvia_Esthefania_Villacres_Chico', 'Thais_Eliana_Armijos_Troya']
def predecir_img(modelo, roi):
    dato = np.array([roi])
    pred = modelo.predict(dato, verbose=False)
    indice = np.argmax(pred[0])
    return clases[indice]

In [14]:
camara = cv2.VideoCapture(0)
ancho_deseado = 500
alto_deseado = 500

marco = 300
margen = (ancho_deseado - marco) // 2
coord_x1 = margen
coord_y1 = margen
coord_x2 = ancho_deseado - margen
coord_y2 = alto_deseado - margen
color_marco = (200, 0, 0)

while True:
    ret, frame = camara.read()
    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (ancho_deseado, alto_deseado))
    cv2.rectangle(frame, (margen, margen), (coord_x2, coord_y2), color_marco, 2)
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    frame_grises = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    roi = frame_grises[coord_y1:coord_y2, coord_x1:coord_x2]
    roi = cv2.resize(roi, (100, 100))
    prediccion = predecir_img(modelo, roi)
    print(f'clase: {prediccion}', end='\r')
    
camara.release()
cv2.destroyAllWindows()

In [12]:
camara.release()
cv2.destroyAllWindows()

In [5]:
clases = ['Abraham_Danilo_Miranda_Lopez', 'andres', 'mateo']
def obtener_nombre(pred):
    # if pred[1] < 115:
    #     return 'desconocido'
    return clases[pred[0]]

In [9]:
# modelo = cv2.face.LBPHFaceRecognizer_create()
# modelo.read('rostros.xml')

In [20]:
import cv2
camara = cv2.VideoCapture(0)
casc_path = './haarcascade_frontalface_default.xml'
dimension_img = 100
while True:
    ret, frame = camara.read()
    frame = cv2.flip(frame, 1)
    face_cascade = cv2.CascadeClassifier(casc_path)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=3,
        minSize=(dimension_img, dimension_img),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 150, 0), 1)
        roi = gray[y-10:y+h+10 , x-10:x+w+10]
        if roi.shape[0] > 0 and roi.shape[1] > 0:
            roi = cv2.resize(roi, (dimension_img,dimension_img))
            cv2.imshow('ROI', roi)
            # pred = modelo.predict(roi)
            # nombre = obtener_nombre(pred)
            nombre = predecir_img(modelo, roi)
            cv2.putText(frame, nombre, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 150, 0), 2)
            

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    cv2.imshow('Rostros', frame)
camara.release()
cv2.destroyAllWindows()